In [1]:
"""ATE: Automatic Term Extraction
"""
import re
import os
import collections
import pandas as pd
from pprint import pprint

from termextract import LRValue
from termextract.preprocessing import extract_nouns, morphemes_to_surface, MeCabTokenizer


In [2]:
file_path = "datasets/生体医工学会論文5755件.csv"
df = pd.read_csv(file_path,encoding='utf-8')

In [3]:
df[df['Published_Year']==2008][['Title','Vol', 'No']].head()

,Title,Vol,No
223,適応ファジィ推論ニューラルネットワークを用いた視覚障害者用学習型位置情報提供システム,46,1
224,記述言語を用いた細胞生理学実験シミュレーション手法,46,1
225,混合脂質膜の透過光スペクトル特性とその自己相関関数に基づく味識別,46,1
226,筋芽細胞の分化と細胞膜電位の変化,46,1
227,生体歩行計測に基づく足首駆動歩行シミュレーション,46,1


In [4]:
years = df["Published_Year"].unique()
years

array([2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015,
       2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023])

In [5]:
docs = {}
for y in years:
    docs[y] = df[(df['Published_Year']==y)]['Title'].values.tolist()

In [19]:
from collections import defaultdict

def intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))
    
tok = MeCabTokenizer() 
use_dic = tok.use_dic

year = 2023
print("\n".join(docs[year][:15]))

compound_nouns = []
docfreq = {}
ignores = ['研究','開発','検討','試作','提案','紹介','展望','事例','課題']
for sentence in docs[year]:
    tokens = tok.iter_token(sentence)
    nouns = extract_nouns(tokens, stopwords=ignores, n=2)
    # print(nouns)
    if len(nouns) > 0 :
        compound_nouns.extend(nouns)

lr = LRValue()
lr_values = lr.fit_transform(compound_nouns)
lr_values = [(k,v) for k,v in lr_values.items() if v>0]
data_collection = sorted(lr_values, key=lambda x:x[1], reverse=True)

生体医工学ウェブ辞典（第二分冊）
医療機関等への建築ガイドライン浸透のための課題と今後の展望
メーカーから見た医用テレメータ設置・運用に関する現状と確認事項
医用テレメータの概要と医療現場での現状と希望
医療機関における電波利用機器に配慮した建築ガイドライン・同解説－医用テレメータ編－のポイント
生体医工学×医療者教育―効率的な知識・技術発展のためにー
生体医工学×知のデザイン －臨床研究における学際の必要性－
生体医工学×医工連携～自身の開発経験談～
生体医工学×医療機器管理のエビデンス創設を目指して
スマートコントラクトを用いたトラストデータ流通システム
聴診音解析システムを活用した医療DXへの挑戦
看護理工学的思考によるリアルワールドデータの活用
皮膚感覚メカニズム解明のためのセンシング・ディスプレイ技術
神経オルガノイドを用いた特殊培養デバイスによる疼痛評価システム
次世代網膜刺激型人工視覚デバイスの開発


In [20]:
list(data_collection)[20:60]

[('乳房 MRI 画像', 16.552849867091567),
 ('刺激 画像', 16.474112694453908),
 ('侵襲 レーザー 皮膚 治療', 16.315893360731785),
 ('計測 解析', 16.186178248771544),
 ('血管内皮 細胞', 15.751599205719526),
 ('光 計測', 15.004664490468443),
 ('支援 システム', 14.982413540254958),
 ('生体 内 機能 評価', 14.79955033479168),
 ('推定 法', 14.660895354719873),
 ('侵襲 肺コンプライアンス 計測 法', 14.464373225885682),
 ('手術 支援 システム', 14.233176803048947),
 ('手術 支援', 14.208401614436143),
 ('細胞 システム', 14.031236411563858),
 ('CT 画像', 13.526840288246285),
 ('血管 検出 法', 13.522856648126485),
 ('画像 脈 波', 13.514795324330045),
 ('および 評価', 13.094437062921891),
 ('筋 活動', 13.091763910376187),
 ('筋 電 計測', 12.904973472680162),
 ('定量 化', 12.872393771853929),
 ('手 指', 12.844170857568436),
 ('細胞 機能', 12.838740166897853),
 ('血圧 自動 計測', 12.594838982460912),
 ('細胞 解析', 12.536440342258926),
 ('および 筋 線維', 12.530410324049738),
 ('3次元 心房 モデル 上', 12.402553055828637),
 ('自動 検出', 12.38395397687857),
 ('電気 的 特性', 12.285401797072755),
 ('パーキンソン病 患者', 12.0),
 ('感情 評価', 11.915784271057897

In [21]:
list(lr.compound_stat().items())[:10]

[('生体 医 工学 ウェブ 辞典', 1),
 ('医療機関 等', 1),
 ('建築 ガイドライン 浸透', 1),
 ('医用 テレメータ 設置', 1),
 ('確認 事項', 1),
 ('医用 テレメータ', 1),
 ('医療 現場', 1),
 ('電波 利用 機器', 1),
 ('建築 ガイドライン', 1),
 ('医用 テレメータ 編', 1)]